In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()

roi = config.roi
data_folder = config.data_folder

lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select([f"classification_{year}" for year in config.range_1985_2020])
        .byte()
        .rename([str(year) for year in config.range_1985_2020]))

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()

method = 'mapbiomas'

folder = method

In [45]:
mature_biomass = ee.Image(f"{data_folder}/mapbiomas/mature_biomass_500")

# Load the individual feature collections
NE_outline = ee.FeatureCollection(f"{data_folder}/NE_outline_no_overlap").map(lambda f: f.set('sector', 1))
NW_outline = ee.FeatureCollection(f"{data_folder}/NW_outline_no_overlap").map(lambda f: f.set('sector', 2))
SW_outline = ee.FeatureCollection(f"{data_folder}/SW_outline_no_overlap").map(lambda f: f.set('sector', 3))
SE_outline = ee.FeatureCollection(f"{data_folder}/SE_outline_no_overlap").map(lambda f: f.set('sector', 4))

# Function to get mean biomass per FeatureCollection
def get_mean_biomass(featureCollection):
    return mature_biomass.reduceRegions(
        reducer=ee.Reducer.mean(),
        collection=featureCollection,
        scale=500  # Adjust this based on your image resolution
    )

# Get mean biomass for each region
mean_biomass_NE = get_mean_biomass(NE_outline)
mean_biomass_NW = get_mean_biomass(NW_outline)
mean_biomass_SW = get_mean_biomass(SW_outline)
mean_biomass_SE = get_mean_biomass(SE_outline)

mean_biomass_NE_img = mean_biomass_NE.reduceToImage(["mean"], ee.Reducer.first()).byte().rename("mean_biomass_NE")
mean_biomass_NW_img = mean_biomass_NW.reduceToImage(["mean"], ee.Reducer.first()).byte().rename("mean_biomass_NW")
mean_biomass_SW_img = mean_biomass_SW.reduceToImage(["mean"], ee.Reducer.first()).byte().rename("mean_biomass_SW")
mean_biomass_SE_img = mean_biomass_SE.reduceToImage(["mean"], ee.Reducer.first()).byte().rename("mean_biomass_SE")

mean_biomass_quarter = mean_biomass_NE_img.addBands([mean_biomass_NW_img, mean_biomass_SW_img, mean_biomass_SE_img]).reduce('sum').byte().rename("mean_biomass_quarter")


merged_outline = NE_outline.merge(NW_outline).merge(SW_outline).merge(SE_outline)
amazon_quarter_regions = merged_outline.reduceToImage(["sector"], ee.Reducer.first()).byte().rename("amazon_quarter_regions")


# export_image(mean_biomass_quarter, "mean_biomass_quarter", region = roi, scale = 1000)
# export_image(amazon_quarter_regions, "amazon_quarter_regions", region = roi, scale = 1000)

map = geemap.Map()
# map.addLayer(mature_biomass, {'min':1, 'max':400, 'palette': ['yellow', 'blue']}, "mature_biomass")
# map.addLayer(NE_outline, {}, "NE_outline", opacity=0.5)
# map.addLayer(NW_outline, {}, "NW_outline", opacity=0.5)
# map.addLayer(SW_outline, {}, "SW_outline", opacity=0.5)
map.addLayer(amazon_quarter_regions, {'min':1, 'max':4, 'palette': ['yellow', 'blue']}, "amazon_quarter_regions", opacity=0.5)
# map.addLayer(mean_biomass_quarter, {'min': 225, 'max': 275, 'palette': ['blue', 'red']}, "mean_biomass_quarter")
# map.addLayer(regions_img, {'min': 1, 'max': 4, 'palette': ['red', 'green', 'blue', 'yellow']}, "regions")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [43]:
mature_biomass = ee.Image(f"{data_folder}/mapbiomas/mature_biomass_500")

mature_biomass_10km = mature_biomass.reproject(
    crs=mature_biomass.projection(),  # Use the same projection as the original image
    scale=10000  # Set the resolution to 10 km (10000 meters)
)

mature_biomass_neighborhood = mature_biomass_10km.reduceNeighborhood(
    reducer=ee.Reducer.mean(),  # Calculate the mean within the neighborhood
    kernel=ee.Kernel.circle(70)
)

# map = geemap.Map()
# map.addLayer(mature_biomass_neighborhood, {'min': 1, 'max': 400, 'palette': ['yellow', 'blue']}, "mature_biomass_neighborhood")
# map
export_image(mature_biomass_neighborhood, "mature_biomass_neighborhood", region = roi, scale = 10000)


## Surrounding mature forests

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Mean fragmentation of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1)

In [18]:
def get_mature_mask():
    if method == 'eu':
        annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)
        return annual_changes.select('Dec2020').eq(1).selfMask()
    elif method == 'mapbiomas':
        return lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()

def get_mature_biomass(biomass_raw, mature_mask):
    if method == 'eu':
        return biomass_raw.updateMask(mature_mask).rename("mature_biomass")
    elif method == 'mapbiomas':
        # keep only forests that are 1000m away from any non-forest area
        edge_detec = mature_mask.unmask(-1).zeroCrossing()
        distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
            .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
        distance_to_edge = distance_to_edge.gt(1000).selfMask().updateMask(mature_mask)
        return biomass_raw.updateMask(distance_to_edge).rename("mature_biomass")

def export_mature_biomass():
    age = ee.Image(f"{data_folder}/{folder}/secondary").select("age")
    biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
    proj = biomass_raw.projection().getInfo()

    mature_mask = get_mature_mask()
    sur_cover = mature_mask.unmask(0).focalMean(radius = 1000, units = "meters").float().updateMask(age).rename("sur_cover")
    mature_biomass = get_mature_biomass(biomass_raw, mature_mask)

    export_image(mature_biomass, "mature_biomass", region = roi, folder = folder, crsTransform = proj['transform'])
    export_image(mature_biomass, "mature_biomass_500", region = roi, folder = folder, scale = 500)
    export_image(sur_cover, "sur_cover", region = roi, folder = folder, scale = 30)

    return mature_biomass

export_mature_biomass()

In [27]:
def get_mature_biomass_vector(mature_biomass):
    # there are time out issues with larger mature_biomass images.
    # to get around it, we reduce to vectors over the geometries of the biomes we are considering
    # and we exclude the pixels that are within the given search radius of the biome edges
    if not panama:
        biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
        amaz_atla_pant = biome.filter(ee.Filter.inList('CD_Bioma', [1, 4, 6]))
        mature_biomass = mature_biomass.clip(amaz_atla_pant)

    mature_biomass_vector = mature_biomass.sample(region = mature_biomass.geometry(), scale = 1000, geometries = True)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = mature_biomass_vector,
        assetId = f"{data_folder}/{folder}/mature_biomass_vector_neighborhood",
        description = "mature_biomass_vector"
    )
        
    task.start()

# mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass")
mature_biomass = ee.Image("projects/amazon-forest-regrowth/assets/mature_biomass_neighborhood")

get_mature_biomass_vector(mature_biomass)


In [6]:
def get_distance_to_nearest_mature(method):

    # secondary = ee.Image(f"{data_folder}/{folder}/secondary").select("age")

    mature_biomass_vector = ee.FeatureCollection(f"{data_folder}/{folder}/mature_biomass_vector_neighborhood")
    # change radius depending on data resolution and patterns
    distance_to_nearest_mature = mature_biomass_vector.distance(searchRadius = 10000)\
        .toInt()

    export_image(distance_to_nearest_mature, "distance_to_nearest_mature_neighborhood", \
                 region = roi, folder = method, scale = 1000)

    return distance_to_nearest_mature

distance_to_nearest_mature = get_distance_to_nearest_mature(method)


In [2]:
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000)))
    return buffered

def get_distances_vector_buffer(distance_to_nearest_mature, method):
    
    if panama:
        distance_to_nearest_mature = distance_to_nearest_mature \
        .addBands(distance_to_nearest_mature.gt(0).selfMask().rename("biome"))
    else:
        categorical = ee.Image(f"{data_folder}/categorical")
        biome_mask = categorical.select('biome').eq(1) \
        .Or(categorical.select('biome').eq(4)) \
        .Or(categorical.select('biome').eq(6))

        distance_to_nearest_mature = distance_to_nearest_mature \
            .addBands(biome_mask)

    distances_vector = distance_to_nearest_mature.stratifiedSample(
        numPoints = 50000, classBand = "biome", region = roi, geometries = True
    )

    distances_vector_buffer = distances_vector.map(buffer_distance)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = distances_vector_buffer,
        assetId = f"{data_folder}/{folder}/distances_vector_buffer",
        description = "distances_vector_buffer"
    )

    task.start()

distance_to_nearest_mature = ee.Image(f"{data_folder}/{folder}/distance_to_nearest_mature_neighborhood")
get_distances_vector_buffer(distance_to_nearest_mature, method)

In [29]:
def export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method):

    nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                            crs = proj['crs'], crsTransform = proj['transform'])

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = nearest_mature_biomass,
        assetId = f"{data_folder}/{folder}/nearest_mature_biomass_neighborhood",
        description = "nearest_mature_biomass"
    )

    task.start()

# mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass_500")
mature_biomass = ee.Image("projects/amazon-forest-regrowth/assets/mature_biomass_neighborhood")
distances_vector_buffer = ee.FeatureCollection(f"{data_folder}/{folder}/distances_vector_buffer")
export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method)

In [34]:
def export_nearest_mature_biomass_image(nearest_mature_biomass, method):

    # change geometries from polygons to points
    nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

    # Convert the FeatureCollection to an image
    nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
        properties = ['first'],
        reducer = ee.Reducer.first()
    ).toInt16().rename('nearest_mature_biomass')

    export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image_neighborhood", \
                 region = roi, folder = method, scale = 1000)

nearest_mature_biomass = ee.FeatureCollection(f"{data_folder}/{folder}/nearest_mature_biomass_neighborhood")
export_nearest_mature_biomass_image(nearest_mature_biomass, method)


In [ ]:
assetId = 'users/your-username/your-asset-name'
ee.data.deleteAsset(assetId)